In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_dir = './drive/MyDrive/ml_class_group_project/Vijay/bert-fine-tuned-ner/checkpoint-5268'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_dir)


In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_dir)

In [ ]:
import torch

def predict(text:str):

  inputs = tokenizer(text, return_tensors="pt")


  ids_list = inputs['input_ids']
  tokens = inputs.tokens()

  # print(type(ids_list[0][3].item()))
  # print(IntTensor.item(ids_list[0][3]))
  # print(inputs.tokens())

  # for id in tf.unstack(ids_list):

  #   print(id)

  # print()


  # print(inputs.input_ids())
  # inputs.decode()

  with torch.no_grad():

    logits = model(**inputs).logits


    predictions = torch.argmax(logits, dim=2)


    predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

  # return (tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True), predicted_token_class)
  # return (text, skip_special_tokens=True), predicted_token_class)

  # return (text, predicted_token_class)

  return (tokens, ids_list, predicted_token_class)

In [ ]:
def merge_subtokens_and_update_labels_list(result_from_predict_function):

  tokens_list = result_from_predict_function[0]
  ids_list = result_from_predict_function[1].tolist()[0]
  labels_list = result_from_predict_function[2]

  # print(type(tokens_list))
  # print(type(ids_list))
  # print(type(labels_list))

  # print(tokens_list)
  # print(ids_list)
  # print(labels_list)

  length_of_labels_list = len(labels_list)

  index = 0

  all = []

  while index < length_of_labels_list:

    current_index = index
    group = []

    while current_index < length_of_labels_list and 'ORG' in labels_list[current_index]:

      group.append(ids_list[current_index])
      current_index += 1


    if len(group) > 0:

      all.append(tokenizer.decode(group))

    if current_index == index:

      index += 1

    else:

      index = current_index


  return all




In [ ]:
# predict('The Golden State Warriors are an American professional basketball team based in San Francisco.')
# predict('Microsoft is a company')

# result_from_predict_function = predict('Desectorization is a company')

# merge_subtokens_and_update_labels_list(result_from_predict_function)

print(merge_subtokens_and_update_labels_list(predict('Hewlett Packard Enterprise is a company.')))

print(merge_subtokens_and_update_labels_list(predict('Mukesh Ambani is the chairman of Reliance Industries Limited')))

print(merge_subtokens_and_update_labels_list(predict('Mukesh Ambani is the chairman of Reliance Industries Limited')))

merge_subtokens_and_update_labels_list(predict('Amazon and Tesla are currently the best picks out there'))



['Hewlett Packard Enterprise']
['Reliance Industries Limited']
['Reliance Industries Limited']


['Amazon', 'Tesla']

In [ ]:
#Function to handle any type of entity

def get_entities(text, entity):

  # text = re.sub(pattern='[^\w\s]', repl='', string=text)
  # print(text)
  # print(entity)

  labels_list = predict(text)[1]

  tokens_list = text.split()
  entities_list = list()

  # print(labels_list)
  # print(tokens_list)
  # print(len(labels_list) == len(tokens_list))

  pairings_list = tuple(zip(tokens_list, labels_list))
  # print(pairings_list)

  index = 0
  length_of_pairings_list = len(pairings_list)

  while index < length_of_pairings_list:

    current_index = index
    new_entity_name = ''

    while current_index < length_of_pairings_list and entity in pairings_list[current_index][1]:

      new_entity_name += pairings_list[current_index - 1][0] + ' '
      current_index += 1

    if len(new_entity_name) > 0:

      new_entity_name = new_entity_name.strip()

      entities_list.append(new_entity_name)

    index += 1 if index == current_index else current_index

  return entities_list if len(entities_list) > 0 else 'No entities found'


In [ ]:
# Single Word Organization Tests

entities = get_entities('Microsoft and Google are large tech companies', 'ORG')
print(entities)

entities = get_entities('Apple and Amazon are competing with each other', 'ORG')
print(entities)

entities = get_entities('Spring has sprung at Amazon', 'ORG')
print(entities)

entities = get_entities('George is a student at university', 'PER')
print(entities)

entities = get_entities('Johnny delivers pizza in New York.', 'PER')
print(entities)

entities = get_entities('German is a language spoken in Germany', 'MISC')
print(entities)

# The below example doesn't work due to subword tokenization

entities = get_entities('Hewlett Packard Enterprise is a company.', 'ORG')
print(entities)


entities = get_entities('Mukesh Ambani is the chairman of Reliance Industries Limited', 'ORG')

entities = get_entities('amazon and tesla are currently the best picks out there', 'ORG')

# Multiple Word Organization Tests

entities = get_entities('British Airways is better than American Airlines.', 'ORG')
print(entities)

entities = get_entities('The United Nations and the World Bank are well known organizations.', 'ORG')
print(entities)

entities = get_entities('The World Trade Organization is an intergovernmental organization headquartered in Geneva, Switzerland.', 'ORG')
print(entities)

entities = get_entities('Johnny delivers pizza in New York.', 'LOC')
print(entities)


['Microsoft', 'Google']
['Apple', 'Amazon']
No entities found
['George']
['Johnny']
['German']
['Hewlett Packard Enterprise is a']
['British Airways', 'American']
['United Nations', 'World Bank']
['World Trade Organization']
['New']


In [ ]:
def mapping(words, labels):

  line1 = ""
  line2 = ""

  print(words.split())
  print(f'Words length is {len(words.split())}')
  print(labels)
  print(f'Labels length is {len(labels)}')

#   for word, label in zip(words, labels):
#       # full_label = label_names[label]
#       max_length = max(len(word), len(full_label))
#       line1 += word + " " * (max_length - len(word) + 1)
#       line2 += full_label + " " * (max_length - len(full_label) + 1)

# print(line1)
# print(line2)

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# def get_organizations(news_article:str):

#   req = requests.get(news_article)
#   soup = BeautifulSoup(req.content, parser='html')

#   content = soup.get_text()

#   tokens = content.split()

#   for token in tokens:
#     print(token)
#     print(predict(token))
#     print()

#   return tokens







#Original one that just gets the word
def get_organizations(text):

  # call predict and store result in list
  # convert text into tokens using split() and store in list
  # iterate through labels list starting from index 1
  # if tag is 'B-ORG', then go to index - 1 in tokens list
  # Append that word into a set
  # return set back to caller

  labels_list = predict(text)
  # print(labels_list)
  tokens_list = text.split()
  organizations_list = set()

  for i in range(1,len(labels_list)):

    if labels_list[i] == 'B-ORG':

      organizations_list.add(tokens_list[i-1])


  if len(organizations_list) == 0:

    organizations_list.add('No organizations found.')


  return list(organizations_list)


In [ ]:
import re

In [ ]:
# print(predict("Rich and Vijay are students at Northeastern University."))
# print(predict("Microsoft and Google are large tech companies"))
print(predict("British Airways is better than American Airlines."))

('British Airways is better than American Airlines.', ['O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O'])


In [ ]:

def get_organizations_v2(text):

  text = re.sub(pattern='[^\w\s]', repl='', string=text)

  labels_list = predict(text)
  tokens_list = text.split()
  organizations_list = set()

  # print(labels_list)
  # print(tokens_list)

  index = 1

  # n represents the length of the labels_list
  n = len(labels_list)

  while index < n - 1:

    current_index = index
    full_organization_name = ''

    while 'ORG' in labels_list[index]:

      if full_organization_name != '':

        full_organization_name += ' ' + tokens_list[index - 1]

      else:

        full_organization_name = tokens_list[index - 1]



      index += 1


    if full_organization_name != '':

      organizations_list.add(full_organization_name)

    if current_index == index:

      index += 1

  organizations_list = list(organizations_list)

  if len(organizations_list) == 0:

    organizations_list.append('No organizations found.')

  return organizations_list


In [ ]:
def merge_company(tokens, tags):
    generated_tokens = []
    i = 0
    while i < len(tags):
        if tags[i] == "B-COMPANY":
            company_token = [tokens[i]]
            for j in range(i + 1, len(tags)):
                i += 1
                if tags[j] != "X":
                    break
                else:
                    company_token.append(tokens[j][2:])
            generated_tokens.append("".join(company_token))
        else:
            generated_tokens.append(tokens[i])
            i += 1

    return generated_tokens

In [ ]:
# tokenized_text = tokenizer('Desectorization')
# print(tokenized_text)

# print(tokenizer.decode(14177))
# print(tokenizer.decode(20302))
# print(tokenizer.decode(2734))
# print(tokenizer.decode([14177, 20302, 2734]))

# tokenizer.decode(tokenized_text['input_ids'])
# predict('Desectorization')

tokenized_text = tokenizer(['Desectorization', 'Demarcation'], is_split_into_words=True)
print(tokenized_text)




print(tokenizer.decode(14177))
print(tokenizer.decode(20302))
print(tokenizer.decode(2734))

print(tokenizer.decode(3177))
print(tokenizer.decode(7317))
print(tokenizer.decode(14520))

# print(tokenizer.decode(16978))
# print(tokenizer.decode([14177, 20302, 2734, 16978]))


{'input_ids': [101, 14177, 20302, 2734, 3177, 7317, 14520, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
Des
##ector
##ization
De
##mar
##cation


In [ ]:
# Single Word Organization Tests

organizations = get_organizations_v2("Microsoft and Google are large tech companies")
print(organizations)

organizations = get_organizations_v2("Apple and Amazon are competing with each other")
print(organizations)

organizations = get_organizations_v2("Spring has sprung at Amazon")
print(organizations)

# Multiple Word Organization Tests

organizations = get_organizations_v2("British Airways is better than American Airlines.")
print(organizations)

organizations = get_organizations_v2("The United Nations and the World Bank are well known organizations.")
print(organizations)

organizations = get_organizations_v2("The World Trade Organization is an intergovernmental organization headquartered in Geneva, Switzerland.")
print(organizations)


# Needs Some Work

# organizations = get_organizations_v2("Hewlett Packard Enterprise is a company.")
# print(organizations)



# organizations = get_organizations_v2("Mukesh Ambani is the chairman of Reliance Industries Limited")
# print(organizations)


# get_organizations('https://www.forbes.com/sites/zakdoffman/2024/03/21/apple-iphone-15-pro-max-upgrade-vs-samsung-galaxy-s24-ultra-s23/?sh=4bcfef0861d7')
# tokens = get_organizations('https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html')


# del predicted_token_class[0]
  # del predicted_token_class[-1]

  # add special tokens -> remove special tokens
  # tokenizer.decode
  # use small text
  #

  # return tuple(zip(text, predicted_token_class))


['No organizations found.']
['No organizations found.']
['No organizations found.']
['No organizations found.']
['No organizations found.']
['No organizations found.']


In [ ]:
#Testing some code here

# tokens:
tokens = ['[CLS]', 'am', '##az', '##on', 'and', 'te', '##sla', 'are', 'currently', 'the', 'best', 'picks', 'out', 'there']
# tags:
tags = ['X', 'B-COMPANY', 'X', 'X', 'O', 'B-COMPANY', 'X', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

tokens = tokens[1:]
tags = tags[1:]

print(tokens)
print(tags)


['am', '##az', '##on', 'and', 'te', '##sla', 'are', 'currently', 'the', 'best', 'picks', 'out', 'there']
['B-COMPANY', 'X', 'X', 'O', 'B-COMPANY', 'X', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
#my practice of implementing the merge function

def merge_entity(tokens, tags):

  pass


In [ ]:
tokens = merge_company(tokens, tags)
tags = [tag for tag in tags if tag != "X"]

print(tokens)
print(tags)

['amazon', 'and', 'tesla', 'are', 'currently', 'the', 'best', 'picks', 'out', 'there']
['B-COMPANY', 'O', 'B-COMPANY', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(tuple(zip(tokens, tags)))

(('amazon', 'B-COMPANY'), ('and', 'O'), ('tesla', 'B-COMPANY'), ('are', 'O'), ('currently', 'O'), ('the', 'O'), ('best', 'O'), ('picks', 'O'), ('out', 'O'), ('there', 'O'))
